-----

# **`Function Calling Using OpenAI API Key`**


-----

### **1. Import Required Libraries**

In [67]:
import os
import openai
import requests
import json
from dotenv import load_dotenv

### **2. Set Up OpenAI API Key**

In [68]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

#### **Using Rapid API For Weather Updates**

In [69]:
import requests
import http.client

def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
    'x-rapidapi-key': "Enter your creedentials",
    'x-rapidapi-host': "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()

In [70]:
response = get_current_weather("Lahore")
response 

[{'name': 'Lahore', 'place_id': 'lahore', 'adm_area1': 'Punjab', 'adm_area2': 'Lahore', 'country': 'Islamic Republic of Pakistan', 'lat': '31.558N', 'lon': '74.35071E', 'timezone': 'Asia/Karachi', 'type': 'settlement'}, {'name': 'Laoré de Falkoumré', 'place_id': 'laore-de-falkoumre-2229614', 'adm_area1': 'Adamaoua Region', 'adm_area2': None, 'country': 'Republic of Cameroon', 'lat': '7.31667N', 'lon': '12.6E', 'timezone': 'Africa/Douala', 'type': 'settlement'}, {'name': 'Lahore', 'place_id': 'lahore-4768156', 'adm_area1': 'Virginia', 'adm_area2': 'Orange', 'country': 'United States of America', 'lat': '38.19875N', 'lon': '77.96944W', 'timezone': 'America/New_York', 'type': 'settlement'}, {'name': 'Kili Lahor', 'place_id': 'kili-lahor-1172453', 'adm_area1': 'Balochistan', 'adm_area2': 'Loralai', 'country': 'Islamic Republic of Pakistan', 'lat': '30.39561N', 'lon': '68.7627E', 'timezone': 'Asia/Karachi', 'type': 'settlement'}, {'name': 'Lahore', 'place_id': 'lahore-district-1172449', 'ad

[{'name': 'Lahore',
  'place_id': 'lahore',
  'adm_area1': 'Punjab',
  'adm_area2': 'Lahore',
  'country': 'Islamic Republic of Pakistan',
  'lat': '31.558N',
  'lon': '74.35071E',
  'timezone': 'Asia/Karachi',
  'type': 'settlement'},
 {'name': 'Laoré de Falkoumré',
  'place_id': 'laore-de-falkoumre-2229614',
  'adm_area1': 'Adamaoua Region',
  'adm_area2': None,
  'country': 'Republic of Cameroon',
  'lat': '7.31667N',
  'lon': '12.6E',
  'timezone': 'Africa/Douala',
  'type': 'settlement'},
 {'name': 'Lahore',
  'place_id': 'lahore-4768156',
  'adm_area1': 'Virginia',
  'adm_area2': 'Orange',
  'country': 'United States of America',
  'lat': '38.19875N',
  'lon': '77.96944W',
  'timezone': 'America/New_York',
  'type': 'settlement'},
 {'name': 'Kili Lahor',
  'place_id': 'kili-lahor-1172453',
  'adm_area1': 'Balochistan',
  'adm_area2': 'Loralai',
  'country': 'Islamic Republic of Pakistan',
  'lat': '30.39561N',
  'lon': '68.7627E',
  'timezone': 'Asia/Karachi',
  'type': 'settleme

#### **Define JSON Function**

In [71]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

### **3. Set up OpenAI ChatCompleation Function**

In [72]:
from openai import OpenAI
client = OpenAI()

user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages= messages,
)

In [73]:
# now add functionlaity of function calling

user_message="Hi There, What's Temprature of Islamabad,Pakistan"
messages.append({"role": "user", "content":user_message})

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages= messages,
    functions=functions,
)

In [74]:
print(completion.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Islamabad, Pakistan"}', name='get_current_weather'), tool_calls=None)


In [75]:
messages # this shows 2 prompts has been added

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user',
  'content': "Hi There, What's Temprature of Islamabad,Pakistan"}]

In [76]:
response=completion.choices[0].message

In [77]:
# Setup function name

function_name=response.function_call.name
print(function_name)

get_current_weather


In [78]:
# import json

# # Simulated response object
# response = {
#     'function_call': {
#         'arguments': '{"location":"Lahore"}'
#     }
# }
# # Parse the JSON string in arguments
# arguments = json.loads(response['function_call']['arguments'])

# # Access the location
# location = arguments['location']
# print(location)

# Parse the function call arguments
arguments = json.loads(response.function_call.arguments)
location = arguments['location']
print(location)

# Append the assistant's function call to the conversation
messages.append({
    "role": "assistant",
    "content": None,
    "function_call": {
        "name": function_name,
        "arguments": json.dumps(arguments),
    }
})

Islamabad, Pakistan


### **4. Send the info on the function call and function response to GPT**

In [79]:
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [80]:
second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages= messages,
    functions=functions,
) 

In [81]:
# Process the second response
print(second_response.choices[0].message.content)

I wasn't able to retrieve the current temperature for Islamabad, Pakistan. However, you can check a reliable weather website or app for the latest updates. If you have any other questions, feel free to ask!
